In [1]:
import numpy as np
from numpy import linalg
from sympy import *

In [2]:
# Rossenbrock function
# f1(x) = 10*(x2 - x1^2)
# f2(x) = 1 - x1
# x0 = (-1.2, 1)
# f = 0 at (1, 1)
# n = 2, m = 2 13

In [3]:
# Freudenstein and Roth function
# f1(x) = -13 + x1 + ((5 - x2)*x2 - 2)*x2
# f2(x) = -29 + x1 + ((x2 + 1)*x2 - 14)*x2
# x0 = (0.5, -2)
# f = 0 at (5, 4)

In [4]:
# Powell badly scaled function
# f1(x) = 10**4*x1*x2 - 1
# f2(x) = exp(-x1) + exp(-x2) - 1.0001
# x0 = (0, 1)
# f = 0 at (1.098...10**-5, 9.106...)

In [5]:
x1 = symbols('x1')
x2 = symbols('x2')

# f1 = simplify("10*(x2 - x1^2)")
# f2 = sympify("1 - x1")
# x0 = np.array([-3, 2], dtype = 'float')

f1 = simplify("(10**4)*x1*x2 - 1")
f2 = simplify("exp(-x1) + exp(-x2) - 1.0001")
x0 = np.array([0, 1], dtype = 'float')

# f1 = simplify("-13 + x1 + ((5 - x2)*x2 - 2)*x2")
# f2 = simplify("-29 + x1 + ((x2 + 1)*x2 - 14)*x2")
# x0 = np.array([0.5, -2], dtype = 'float')
# x0 = np.array([5, -10], dtype = 'float')

x = np.array([x1, x2])
F = np.array([f1, f2])

In [6]:
F_0 = lambda x0: np.array([F[i].subs(dict(zip(x, x0))) for i in range(len(F))], dtype = 'float')

In [7]:
def F_der_0(x0):
    F_der = np.array([[diff(F[i], x[j]) for j in range(len(x))] for i in range(len(F))])
    return np.array([[F_der[i][j].subs(dict(zip(x, x0))) for j in range(len(F_der[i]))] for i in range(len(F_der))], dtype = 'float')

In [8]:
# test function for F(x, y)
# A = np.array([simplify("x1**2 + 7*x1 + 2*x1*x2 - 3"), simplify("x1 + x2**3")])
A = np.array([simplify("-13 + x1 + ((5 - x2)*x2 - 2)*x2"), simplify("-29 + x1 + ((x2 + 1)*x2 - 14)*x2")])
A[0].subs({x1: 1.13975640, x2: 1.05684169})

-9.56975684878996

In [9]:
def F_1(x1, x2):
    assert(len(x1) == len(x2))
    arr = []
    for i in range(len(F)):
        subarr = []
        for j in range(len(x1)):
            params1 = [*x1[0:j+1], *x2[j+1:len(x2)+1]]
            params2 = [*x1[0:j], *x2[j:len(x2)+1]]
            # print("x1[j] = {0}, x2[j] = {1}, x1[j] - x2[j] = {2}".format(x1[j], x2[j], x1[j] - x2[j]))
            # print((F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))))
            result = (F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))) / (x1[j] - x2[j])
            subarr.append(result)
        
        arr.append(subarr)
    
    return np.array(arr, dtype = 'float')


In [10]:
def NewthonMethod(x0, x, F):
    I = 0
    x1 = x0
    while max(np.abs(F_0(x0))) > 10 ** -7:
        x1 = x0 - np.dot(np.linalg.inv(F_der_0(x0)), F_0(x0))
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I, F_0(x1), x1))
        I += 1

        x0 = x1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [11]:
def PotraMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    I = 0
    while max(np.abs(F_0(x0))) > 10 ** -7:
        matrix = (F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1)
        x1 = x0 - np.dot(np.linalg.inv(matrix), F_0(x0))
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I, F_0(x1), x1))
        I += 1

        x_2 = x_1
        x_1 = x0
        x0 = x1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0


In [12]:
def PotraInverseOperatorMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv((F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1))
    while max(np.abs(F_0(x0))) > 10 ** -7:
        x1 = x0 - np.dot(A_0, F_0(x0))

        matrix = (F_1(x1, x0) + F_1(x_1, x1)) - F_1(x_1, x0)
        # right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
        right_matrix = 2 * I - np.dot(matrix, A_0)
        A_1 = np.dot(A_0, right_matrix)
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations, F_0(x1), x1))
        number_of_iterations += 1

        x_2 = x_1
        x_1 = x0
        x0 = x1
        A_0 = A_1
    
    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0


In [13]:
# Inverse operator methods
def NewthonInverseOperatorMethod(x0, x, F):
    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv(F_der_0(x0))
    while max(np.abs(F_0(x0))) > 10 ** -7:
        x1 = x0 - np.dot(A_0, F_0(x0))
        right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
        A_1 = np.dot(A_0, right_matrix)
        print(F_0(x1))
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations, F_0(x1), x1))
        x0 = x1
        A_0 = A_1
        number_of_iterations += 1

    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0
    

In [14]:
print("Метод Ньютона з послідовною апроксимацією оберненого оператора\n\n")
print(NewthonInverseOperatorMethod(x0, x, F))

Метод Ньютона з послідовною апроксимацією оберненого оператора


[0.99945634 0.13520888]
0 Ітерація: Значення F = [0.99945634 0.13520888] у точці x = [1.00000000e-04 1.99945634e+00]
[0.64198285 0.07414627]
1 Ітерація: Значення F = [0.64198285 0.07414627] у точці x = [6.31649106e-05 2.59951742e+00]
[-0.07357249  0.04275631]
2 Ітерація: Значення F = [-0.07357249  0.04275631] у точці x = [2.94177162e-05 3.14921627e+00]
[-1.11494202  0.02507031]
3 Ітерація: Значення F = [-1.11494202  0.02507031] у точці x = [-3.12154629e-06  3.68221411e+00]
[-2.44870911  0.01480708]
4 Ітерація: Значення F = [-2.44870911  0.01480708] у точці x = [-3.44256062e-05  4.20823123e+00]
[-3.97263524  0.00879197]
5 Ітерація: Значення F = [-3.97263524  0.00879197] у точці x = [-6.28503970e-05  4.72970002e+00]
[-5.39997762e+00  5.25447218e-03]
6 Ітерація: Значення F = [-5.39997762e+00  5.25447218e-03] у точці x = [-8.38791875e-05  5.24561307e+00]
[-6.08655874e+00  3.16228188e-03]
7 Ітерація: Значення F = [-6.08655874e

In [15]:
print("Метод Потра з послідовною апроксимацією оберненого оператора\n\n")
print(PotraInverseOperatorMethod(x0, x, F))

Метод Потра з послідовною апроксимацією оберненого оператора


0 Ітерація: Значення F = [0.99945635 0.13520888] у точці x = [1.00000000e-04 1.99945635e+00]
1 Ітерація: Значення F = [0.66604345 0.07136893] у точці x = [6.31649109e-05 2.63760912e+00]
2 Ітерація: Значення F = [-0.07131619  0.0392569 ] у точці x = [2.87131107e-05 3.23435457e+00]
3 Ітерація: Значення F = [-1.15198326  0.02218455] у точці x = [-3.99531110e-06  3.80404076e+00]
4 Ітерація: Значення F = [-2.53212634  0.01270477] у точці x = [-3.51349912e-05  4.36068515e+00]
5 Ітерація: Значення F = [-4.09925749  0.00734578] у точці x = [-6.31390219e-05  4.90862449e+00]
6 Ітерація: Значення F = [-5.54693477e+00  4.29232174e-03] у точці x = [-8.34746324e-05  5.44708571e+00]
7 Ітерація: Значення F = [-6.19821775e+00  2.53682564e-03] у точці x = [-8.70468058e-05  5.97175014e+00]
8 Ітерація: Значення F = [-5.12610803e+00  1.49869305e-03] у точці x = [-6.36821806e-05  6.47921914e+00]
9 Ітерація: Значення F = [-2.48198239e+00  8.54511

In [16]:
print("Метод Ньютона\n\n")
print(NewthonMethod(x0, x, F))

Метод Ньютона


0 Ітерація: Значення F = [0.99945634 0.13520888] у точці x = [1.00000000e-04 1.99945634e+00]
1 Ітерація: Значення F = [-0.99889195  0.04975092] у точці x = [3.69507336e-08 2.99871753e+00]
2 Ітерація: Значення F = [0.33209287 0.01825493] у точці x = [3.33352999e-05 3.99604285e+00]
3 Ітерація: Значення F = [-0.16494281  0.00669101] у точці x = [1.67356377e-05 4.98969447e+00]
4 Ітерація: Значення F = [8.98994125e-05 2.43100278e-03] у точці x = [1.67447845e-05 5.97254567e+00]
5 Ітерація: Значення F = [-0.02559642  0.00086611] у точці x = [1.40651742e-05 6.92777472e+00]
6 Ітерація: Значення F = [-0.01263437  0.00029186] у точці x = [1.26376987e-05 7.81285944e+00]
7 Ітерація: Значення F = [-7.30875707e-03  8.44703277e-05] у точці x = [1.16282492e-05 8.53689343e+00]
8 Ітерація: Значення F = [-2.18670063e-03  1.60164943e-05] у точці x = [1.11236196e-05 8.97022131e+00]
9 Ітерація: Значення F = [-1.69115210e-04  9.83598807e-07] у точці x = [1.09904815e-05 9.09724368e+00]
10 Ітера

In [17]:
print("Метод Потра\n\n")
print(PotraMethod(x0, x, F))

Метод Потра


0 Ітерація: Значення F = [0.99945635 0.13520888] у точці x = [1.00000000e-04 1.99945635e+00]
1 Ітерація: Значення F = [-1.66244267  0.03362303] у точці x = [-1.9540194e-05  3.3901540e+00]
2 Ітерація: Значення F = [-0.6697155   0.18604516] у точці x = [1.96466595e-05 1.68112292e+00]
3 Ітерація: Значення F = [0.29093732 0.0643628 ] у точці x = [4.70984001e-05 2.74093668e+00]
4 Ітерація: Значення F = [-0.22715268  0.02659198] у точці x = [2.13340913e-05 3.62259310e+00]
5 Ітерація: Значення F = [-0.23637573  0.00186129] у точці x = [1.22613053e-05 6.22791990e+00]
6 Ітерація: Значення F = [-0.00485037  0.00210953] у точці x = [1.62935473e-05 6.10763029e+00]
7 Ітерація: Значення F = [-0.01905104  0.00082773] у точці x = [1.40785059e-05 6.96770642e+00]
8 Ітерація: Значення F = [-0.01938884  0.00021748] у точці x = [1.22312105e-05 8.01728629e+00]
9 Ітерація: Значення F = [-1.01535705e-01 -8.53582896e-05] у точці x = [8.41449597e-06 1.06775771e+01]
10 Ітерація: Значення F = [ 2.51